In [1]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20160101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')
df_d.tail()

select * from stock_day where date>=20160101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(3520947, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3520947 entries, (000001.SZ, 2016-01-04 00:00:00) to (688399.SH, 2020-04-22 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 321.6 MB


open        high         low       close  \
code      date                                                         
688399.SH 2020-04-16   83.400002   96.000000   82.440002   94.900002   
          2020-04-17   95.949997   95.949997   88.879997   92.000000   
          2020-04-20   95.790001  109.970001   95.199997  108.730003   
          2020-04-21  108.000000  118.199997  102.000000  114.440002   
          2020-04-22  108.000000  118.500000  107.489998  109.089996   

                               vol       amt  adj_factor         avg  \
code      date                                                         
688399.SH 2020-04-16  31260.830078  2.815096         1.0   90.051849   
          2020-04-17  24047.009766  2.208113         1.0   91.824860   
          2020-04-20  34636.898438  3.590274         1.0  103.654602   
          2020-04-21  33081.828125  3.580782         1.0  108.240166   
          2020-04-22  21584.169922  2.432959         1.0  112.719612   

                          close0        avg0        pct  
code      date                                           
688399.SH 2020-04-16   94.900002   90.051849  15.731705  
          2020-04-17   92.000000   91.824860  -3.055853  
          2020-04-20  108.730003  103.654602  18.184780  
          2020-04-21  114.440002  108.240166   5.251539  
          2020-04-22  109.089996  112.719612  -4.674941

In [4]:
df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
#
df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

df_d_basic = df_d_basic.astype('float32')
print(df_d_basic.columns)
print(df_d_basic.shape)
df_d_basic.info(memory_usage='deep')
df_d_basic.tail()

select * from stock_daily_basic where date>=20160101
Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(3363008, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3363008 entries, (000001.SZ, 2016-01-04 00:00:00) to (688399.SH, 2020-04-22 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 218.3 MB


close  turnover_rate  turnover_rate_f  \
code      date                                                     
688399.SH 2020-04-16   94.900002      23.436600        23.436600   
          2020-04-17   92.000000      18.028299        18.028299   
          2020-04-20  108.730003      25.967699        25.967699   
          2020-04-21  114.440002      24.801800        24.801800   
          2020-04-22  109.089996      16.181900        16.181900   

                      volume_ratio          pe     pe_ttm      pb         ps  \
code      date                                                                 
688399.SH 2020-04-16          3.37   87.161400  62.353401  5.4207  24.113701   
          2020-04-17          1.74   84.497902  60.448002  5.2551  23.376801   
          2020-04-20          2.14   99.863602  71.440300  6.2107  27.627800   
          2020-04-21          1.54  105.108002  75.192001  6.5369  29.078699   
          2020-04-22          0.81  100.194298  71.676804  6.2313  27.719299   

                         ps_ttm  total_share  float_share   free_share  \
code      date                                                           
688399.SH 2020-04-16  19.117100       5862.0  1333.847046  1333.847046   
          2020-04-17  18.532900       5862.0  1333.847046  1333.847046   
          2020-04-20  21.903099       5862.0  1333.847046  1333.847046   
          2020-04-21  23.053400       5862.0  1333.847046  1333.847046   
          2020-04-22  21.975599       5862.0  1333.847046  1333.847046   

                         total_mv        circ_mv   pb*pe_ttm       pb*pe  
code      date                                                            
688399.SH 2020-04-16  556303.8125  126582.078125  337.999084  472.475800  
          2020-04-17  539304.0000  122713.921875  317.660278  444.044891  
          2020-04-20  637375.2500  145029.187500  443.694275  620.222900  
          2020-04-21  670847.2500  152645.453125  491.522583  687.080505  
          2020-04-22  639485.5625  145509.375000  446.639648  624.340698

In [6]:
df = df_d.join(df_d_basic.drop(columns=['close']),how='left')
periods = [5,10,20,30,40,60,80,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
    df['{:d}ma_avg'.format(k)]=df.reset_index('code').groupby('code')['amt'].rolling(k).sum()\
            /df.reset_index('code').groupby('code')['vol'].rolling(k).sum()*1e6
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]

for col2 in ["{}ma_avg".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]

print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', 'turnover_rate', 'turnover_rate_f',
       'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm',
       'pb*pe', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma', '20ma_avg',
       '30ma', '30ma_avg', '40ma', '40ma_avg', '60ma', '60ma_avg', '80ma',
       '80ma_avg', '120ma', '120ma_avg', '250ma', '250ma_avg'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', 'turnover_rate', 'turnover_rate_f',
       'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm',
       'pb*pe', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma', '20ma_avg',
       '30ma', '30ma_avg', '40ma', '40ma_avg', '60ma', '60ma_avg', '80ma',
       '80ma_avg', '120ma', '120ma_a

In [8]:
# df = df[].groupby('code').rolling(win,min_periods=int(win*0.9)).quantile(0.01)
col_pair = []
for k in [5,10,20,60,120]:
    col1 = 'close'
    col2 = '{}ma'.format(k)
    col3 = '{}ma_avg'.format(k)
    col_pair.extend([(col1,col2),(col1,col3)])
cols = ['{}/{}'.format(col1,col2) for col1,col2 in col_pair]    

['close/5ma',
 'close/5ma_avg',
 'close/10ma',
 'close/10ma_avg',
 'close/20ma',
 'close/20ma_avg',
 'close/60ma',
 'close/60ma_avg',
 'close/120ma',
 'close/120ma_avg']

In [18]:
win = 500
q=0.01
col = 'close/20ma'
col
# df['{}_q{:.0%}'.format(col,q)] = df[col].groupby('code').rolling(win,min_periods=int(win*0.9)).quantile(q)
df.reset_index('code').groupby('code')[col].rolling(win,min_periods=int(win*0.9)).quantile(q)

'close/20ma'

code       date      
000001.SZ  2016-01-04   NaN
           2016-01-05   NaN
           2016-01-06   NaN
           2016-01-07   NaN
           2016-01-08   NaN
                         ..
688399.SH  2020-04-16   NaN
           2020-04-17   NaN
           2020-04-20   NaN
           2020-04-21   NaN
           2020-04-22   NaN
Name: close/20ma, Length: 3520947, dtype: float64

In [ ]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r_spl = pd.read_parquet(r"database\return_spl_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()
df_r_spc = pd.read_parquet(r"database\return_spc_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()